# Processing routine

+ The following script outlines the steps to ingest acquired metadata (ScanImage and Scanbox) and processed data (Suite2p and CaImAn) into the DataJoint `workflow-calcium-imaging`.

+ To ingest with a completely automated workflow, see `03-automate[optional].ipynb`.

In [2]:
# Change into the parent directory to find the `dj_local_conf.json` file. 
# When you type `import datajoint as dj` the credentials from `dj_local_conf.json` will be used to log into the database.

import os
os.chdir('..')

In [ ]:
import numpy as np
from workflow_calcium_imaging.pipeline import *

## Schema diagrams

+ The following outputs are the diagrams of the schemas comprising this workflow.

+ Please refer back to these diagrams to visualize the relationships of different tables.

In [ ]:
dj.Diagram(lab)

In [ ]:
dj.Diagram(subject)

In [ ]:
dj.Diagram(session)

In [ ]:
dj.Diagram(scan)

In [ ]:
dj.Diagram(imaging)

## Insert an entry into `subject.Subject`

In [ ]:
subject.Subject.insert1(dict(subject='subject1', 
                             sex='F', 
                             subject_birth_date='2019-01-01 00:00:01', 
                             subject_description='no description'))

## Insert an entry into `lab.Equipment`

In [ ]:
Equipment.insert1(dict(scanner='Miniscope-DAQ-V3'))

## Insert an entry into `session.Session`

In [ ]:
session.Session.insert1(dict(subject='subject1', 
                             session_datetime='2021-01-01 00:00:01'))

## Insert an entry into `session.SessionDirectory`

In [ ]:
session.SessionDirectory.insert1(dict(subject='subject1', 
                                      session_datetime='2021-01-01 00:00:01', 
                                      session_dir='<imaging_root_data_dir>/subject1/session0'))

## Insert an entry into `scan.Scan`

In [ ]:
scan.Scan.insert1(dict(subject='subject1', 
                       session_datetime='2021-01-01 00:00:01', 
                       scan_id=0, 
                       scanner='Miniscope-DAQ-V3', 
                       acq_software='Miniscope-DAQ-V3',
                       scan_notes=''))

## Populate `scan.ScanInfo`

+ This imported table stores information about the acquired image (e.g. image dimensions, file paths, etc.).
+ `populate` automatically calls `make` for every key for which the auto-populated table is missing data.
+ `populate_settings` passes arguments to the `populate` method.
+ `display_progress=True` reports the progress bar

In [ ]:
populate_settings = {'display_progress': True}

In [ ]:
scan.ScanInfo.populate(**populate_settings)

## Insert an entry into `imaging.ProcessingTask`

+ This entry will trigger ingestion of the processed results (i.e. motion correction, segmentation, and traces)

+ The `paramset_idx` is the parameter set stored in `imaging.ProcessingParamSet` that is used for the image processing.

+ The `processing_output_dir` attribute contains the output directory of the processed results (relative the the imaging root data directory).

In [ ]:
imaging.ProcessingTask.insert1(dict(subject='subject1', 
                                    session_datetime='2021-01-01 00:00:01', 
                                    scan_id=0,
                                    paramset_idx=0,
                                    processing_output_dir='<imaging_root_data_dir>/subject1/session0/miniscope_analysis',
                                    task_mode='load'))

## Populate `imaging.Processing`

+ For the `task_mode=load` specified above in `imaging.ProcessingTask`, this step ensures that the output directory contains the valid processed outputs.

+ In the future, this step will provide for the option to `trigger` the analysis within this workflow (if the `task_mode=trigger`).

In [ ]:
imaging.Processing.populate(**populate_settings)

## Populate `imaging.MotionCorrection`

+ This table contains the rigid or non-rigid motion correction data including the shifts and summary images.


In [ ]:
imaging.MotionCorrection.populate(**populate_settings)

## Insert an entry into `imaging.Curation`

+ The next step in the pipeline is the curation of segmentation results. If a manual curation was implemented, an entry needs to be manually inserted into the table Curation, which specifies the directory to the curated results in curation_output_dir. If we would like to process the processed outcome directly, an entry is also needed in Curation. A method create1_from_processing_task was provided to help this insertion. It copies the processing_output_dir in ProcessingTask to the field curation_output_dir in the table Curation with a new curation_id.

+ In this example, we create/insert one `imaging.Curation` for each `imaging.ProcessingTask`, specifying the same output directory.

In [ ]:
imaging.Curation(dict(subject='subject1', 
                      session_datetime='2021-01-01 00:00:01', 
                      scan_id=0,
                      paramset_idx=0,
                      curation_id=0,
                      curation_time='2021-01-01 00:00:01', 
                      curation_output_dir='<imaging_root_data_dir>/subject1/session0/miniscope_analysis',
                      manual_curation=False,
                      curation_note=''})

## Populate `imaging.Segmentation`

+ This table contains the mask coordinates, weights, and centers.

In [ ]:
imaging.Segmentation.populate(**populate_settings)

## Populate `imaging.MaskClassification`

+ This table is currently not implemented.

In [ ]:
imaging.MaskClassification.populate(**populate_settings)

## Populate `imaging.Fluorescence`

+ This table contains the fluorescence traces prior filtering and spike extraction

In [ ]:
imaging.Fluorescence.populate(**populate_settings)

## Populate `imaging.Activity`
+ This table contains the inferred neural activity from the fluorescence traces.

In [ ]:
imaging.Activity.populate(**populate_settings)

## Proceed to the `04-explore.ipynb` Jupyter Notebook

+ This notebook describes the steps to query, fetch, and visualize the imaging data.

## Insert parameters for Suite2p and CaImAn into the following table:
+ imaging.ProcessingParamSet

### Define Suite2p parameters

In [2]:
params_suite2p = {'look_one_level_down': 0.0,
 'fast_disk': [],
 'delete_bin': False,
 'mesoscan': False,
 'h5py': [],
 'h5py_key': 'data',
 'save_path0': [],
 'subfolders': [],
 'nplanes': 1,
 'nchannels': 1,
 'functional_chan': 1,
 'tau': 1.0,
 'fs': 10.0,
 'force_sktiff': False,
 'preclassify': 0.0,
 'save_mat': False,
 'combined': True,
 'aspect': 1.0,
 'do_bidiphase': False,
 'bidiphase': 0.0,
 'do_registration': True,
 'keep_movie_raw': False,
 'nimg_init': 300,
 'batch_size': 500,
 'maxregshift': 0.1,
 'align_by_chan': 1,
 'reg_tif': False,
 'reg_tif_chan2': False,
 'subpixel': 10,
 'smooth_sigma': 1.15,
 'th_badframes': 1.0,
 'pad_fft': False,
 'nonrigid': True,
 'block_size': [128, 128],
 'snr_thresh': 1.2,
 'maxregshiftNR': 5.0,
 '1Preg': False,
 'spatial_hp': 50.0,
 'pre_smooth': 2.0,
 'spatial_taper': 50.0,
 'roidetect': True,
 'sparse_mode': False,
 'diameter': 12,
 'spatial_scale': 0,
 'connected': True,
 'nbinned': 5000,
 'max_iterations': 20,
 'threshold_scaling': 1.0,
 'max_overlap': 0.75,
 'high_pass': 100.0,
 'inner_neuropil_radius': 2,
 'min_neuropil_pixels': 350,
 'allow_overlap': False,
 'chan2_thres': 0.65,
 'baseline': 'maximin',
 'win_baseline': 60.0,
 'sig_baseline': 10.0,
 'prctile_baseline': 8.0,
 'neucoeff': 0.7,
 'xrange': np.array([0, 0]),
 'yrange': np.array([0, 0])}

### Insert Suite2p parameters

In [3]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method='suite2p', 
    package_version='0.9.3',
    paramset_idx=0, 
    paramset_desc='Calcium imaging analysis with Suite2p using default Suite2p parameters', 
    params=params_suite2p)

### Define CaImAn 2d planar parameters

In [4]:
params_caiman_2d = {'fnames': None,
 'dims': None,
 'fr': 30,
 'decay_time': 0.4,
 'dxy': (1, 1),
 'var_name_hdf5': 'mov',
 'caiman_version': '1.8.5',
 'last_commit': 'GITW-a99c03c9cb221e802ec71aacfb988257810c8c4a',
 'mmap_F': None,
 'mmap_C': None,
 'block_size_spat': 5000,
 'dist': 3,
 'expandCore': np.array([[0, 0, 1, 0, 0],
        [0, 1, 1, 1, 0],
        [1, 1, 1, 1, 1],
        [0, 1, 1, 1, 0],
        [0, 0, 1, 0, 0]], dtype='int32'),
 'extract_cc': True,
 'maxthr': 0.1,
 'medw': None,
 'method_exp': 'dilate',
 'method_ls': 'lasso_lars',
 'n_pixels_per_process': None,
 'nb': 1,
 'normalize_yyt_one': True,
 'nrgthr': 0.9999,
 'num_blocks_per_run_spat': 20,
 'se': np.array([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]], dtype='uint8'),
 'ss': np.array([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]], dtype='uint8'),
 'thr_method': 'nrg',
 'update_background_components': True,
 'ITER': 2,
 'bas_nonneg': False,
 'block_size_temp': 5000,
 'fudge_factor': 0.96,
 'lags': 5,
 'optimize_g': False,
 'memory_efficient': False,
 'method_deconvolution': 'oasis',
 'noise_method': 'mean',
 'noise_range': [0.25, 0.5],
 'num_blocks_per_run_temp': 20,
 'p': 2,
 's_min': None,
 'solvers': ['ECOS', 'SCS'],
 'verbosity': False,
 'K': 30,
 'SC_kernel': 'heat',
 'SC_sigma': 1,
 'SC_thr': 0,
 'SC_normalize': True,
 'SC_use_NN': False,
 'SC_nnn': 20,
 'alpha_snmf': 100,
 'center_psf': False,
 'gSig': [5, 5],
 'gSiz': (11, 11),
 'init_iter': 2,
 'kernel': None,
 'lambda_gnmf': 1,
 'maxIter': 5,
 'max_iter_snmf': 500,
 'method_init': 'greedy_roi',
 'min_corr': 0.85,
 'min_pnr': 20,
 'nIter': 5,
 'normalize_init': True,
 'options_local_NMF': None,
 'perc_baseline_snmf': 20,
 'ring_size_factor': 1.5,
 'rolling_length': 100,
 'rolling_sum': True,
 'seed_method': 'auto',
 'sigma_smooth_snmf': (0.5, 0.5, 0.5),
 'ssub': 2,
 'ssub_B': 2,
 'tsub': 2,
 'check_nan': True,
 'compute_g': False,
 'include_noise': False,
 'max_num_samples_fft': 3072,
 'pixels': None,
 'sn': None,
 'border_pix': 0,
 'del_duplicates': False,
 'in_memory': True,
 'low_rank_background': True,
 'memory_fact': 1,
 'n_processes': 1,
 'nb_patch': 1,
 'only_init': True,
 'p_patch': 0,
 'remove_very_bad_comps': False,
 'rf': None,
 'skip_refinement': False,
 'p_ssub': 2,
 'stride': None,
 'p_tsub': 2,
 'N_samples_exceptionality': 12,
 'batch_update_suff_stat': False,
 'dist_shape_update': False,
 'ds_factor': 1,
 'epochs': 1,
 'expected_comps': 500,
 'full_XXt': False,
 'init_batch': 200,
 'init_method': 'bare',
 'iters_shape': 5,
 'max_comp_update_shape': np.inf,
 'max_num_added': 5,
 'max_shifts_online': 10,
 'min_SNR': 2.5,
 'min_num_trial': 5,
 'minibatch_shape': 100,
 'minibatch_suff_stat': 5,
 'motion_correct': True,
 'movie_name_online': 'online_movie.mp4',
 'normalize': False,
 'n_refit': 0,
 'num_times_comp_updated': np.inf,
 'opencv_codec': 'H264',
 'path_to_model': None,
 'ring_CNN': False,
 'rval_thr': 0.8,
 'save_online_movie': False,
 'show_movie': False,
 'simultaneously': False,
 'sniper_mode': False,
 'stop_detection': False,
 'test_both': False,
 'thresh_CNN_noisy': 0.5,
 'thresh_fitness_delta': -50,
 'thresh_fitness_raw': -60.97977932734429,
 'thresh_overlap': 0.5,
 'update_freq': 200,
 'update_num_comps': True,
 'use_corr_img': False,
 'use_dense': True,
 'use_peak_max': True,
 'W_update_factor': 1,
 'SNR_lowest': 0.5,
 'cnn_lowest': 0.1,
 'gSig_range': None,
 'min_cnn_thr': 0.9,
 'rval_lowest': -1,
 'use_cnn': True,
 'use_ecc': False,
 'max_ecc': 3,
 'do_merge': True,
 'merge_thr': 0.8,
 'merge_parallel': False,
 'max_merge_area': None,
 'border_nan': 'copy',
 'gSig_filt': None,
 'is3D': False,
 'max_deviation_rigid': 3,
 'max_shifts': (6, 6),
 'min_mov': None,
 'niter_rig': 1,
 'nonneg_movie': True,
 'num_frames_split': 80,
 'num_splits_to_process_els': None,
 'num_splits_to_process_rig': None,
 'overlaps': (32, 32),
 'pw_rigid': False,
 'shifts_opencv': True,
 'splits_els': 14,
 'splits_rig': 14,
 'strides': (96, 96),
 'upsample_factor_grid': 4,
 'use_cuda': False,
 'n_channels': 2,
 'use_bias': False,
 'use_add': False,
 'pct': 0.01,
 'patience': 3,
 'max_epochs': 100,
 'width': 5,
 'loss_fn': 'pct',
 'lr': 0.001,
 'lr_scheduler': None,
 'remove_activity': False,
 'reuse_model': False}

### Insert CaImAn parameters

In [5]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method='caiman', 
    package_version='1.8.7', 
    paramset_idx=1, 
    paramset_desc='Calcium imaging analysis with CaImAn using default CaImAn parameters for 2d planar images', 
    params=params_caiman_2d)

### Define CaImAn 3d volumetric parameters
    strides    = (96, 96, 1)
    overlaps   = (32, 32, 1)
    max_shifts = (6, 6, 1)
    gSig       = (5, 5, 1)
    use_cnn    = False

In [6]:
params_caiman_3d = {'fnames': None,
 'dims': None,
 'fr': 30,
 'decay_time': 0.4,
 'dxy': (1, 1),
 'var_name_hdf5': 'mov',
 'caiman_version': '1.8.5',
 'last_commit': 'GITW-a99c03c9cb221e802ec71aacfb988257810c8c4a',
 'mmap_F': None,
 'mmap_C': None,
 'block_size_spat': 5000,
 'dist': 3,
 'expandCore': np.array([[0, 0, 1, 0, 0],
        [0, 1, 1, 1, 0],
        [1, 1, 1, 1, 1],
        [0, 1, 1, 1, 0],
        [0, 0, 1, 0, 0]], dtype='int32'),
 'extract_cc': True,
 'maxthr': 0.1,
 'medw': None,
 'method_exp': 'dilate',
 'method_ls': 'lasso_lars',
 'n_pixels_per_process': None,
 'nb': 1,
 'normalize_yyt_one': True,
 'nrgthr': 0.9999,
 'num_blocks_per_run_spat': 20,
 'se': np.array([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]], dtype='uint8'),
 'ss': np.array([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]], dtype='uint8'),
 'thr_method': 'nrg',
 'update_background_components': True,
 'ITER': 2,
 'bas_nonneg': False,
 'block_size_temp': 5000,
 'fudge_factor': 0.96,
 'lags': 5,
 'optimize_g': False,
 'memory_efficient': False,
 'method_deconvolution': 'oasis',
 'noise_method': 'mean',
 'noise_range': [0.25, 0.5],
 'num_blocks_per_run_temp': 20,
 'p': 2,
 's_min': None,
 'solvers': ['ECOS', 'SCS'],
 'verbosity': False,
 'K': 30,
 'SC_kernel': 'heat',
 'SC_sigma': 1,
 'SC_thr': 0,
 'SC_normalize': True,
 'SC_use_NN': False,
 'SC_nnn': 20,
 'alpha_snmf': 100,
 'center_psf': False,
 'gSig': (5, 5, 1),
 'gSiz': (11, 11),
 'init_iter': 2,
 'kernel': None,
 'lambda_gnmf': 1,
 'maxIter': 5,
 'max_iter_snmf': 500,
 'method_init': 'greedy_roi',
 'min_corr': 0.85,
 'min_pnr': 20,
 'nIter': 5,
 'normalize_init': True,
 'options_local_NMF': None,
 'perc_baseline_snmf': 20,
 'ring_size_factor': 1.5,
 'rolling_length': 100,
 'rolling_sum': True,
 'seed_method': 'auto',
 'sigma_smooth_snmf': (0.5, 0.5, 0.5),
 'ssub': 2,
 'ssub_B': 2,
 'tsub': 2,
 'check_nan': True,
 'compute_g': False,
 'include_noise': False,
 'max_num_samples_fft': 3072,
 'pixels': None,
 'sn': None,
 'border_pix': 0,
 'del_duplicates': False,
 'in_memory': True,
 'low_rank_background': True,
 'memory_fact': 1,
 'n_processes': 1,
 'nb_patch': 1,
 'only_init': True,
 'p_patch': 0,
 'remove_very_bad_comps': False,
 'rf': None,
 'skip_refinement': False,
 'p_ssub': 2,
 'stride': None,
 'p_tsub': 2,
 'N_samples_exceptionality': 12,
 'batch_update_suff_stat': False,
 'dist_shape_update': False,
 'ds_factor': 1,
 'epochs': 1,
 'expected_comps': 500,
 'full_XXt': False,
 'init_batch': 200,
 'init_method': 'bare',
 'iters_shape': 5,
 'max_comp_update_shape': np.inf,
 'max_num_added': 5,
 'max_shifts_online': 10,
 'min_SNR': 2.5,
 'min_num_trial': 5,
 'minibatch_shape': 100,
 'minibatch_suff_stat': 5,
 'motion_correct': True,
 'movie_name_online': 'online_movie.mp4',
 'normalize': False,
 'n_refit': 0,
 'num_times_comp_updated': np.inf,
 'opencv_codec': 'H264',
 'path_to_model': None,
 'ring_CNN': False,
 'rval_thr': 0.8,
 'save_online_movie': False,
 'show_movie': False,
 'simultaneously': False,
 'sniper_mode': False,
 'stop_detection': False,
 'test_both': False,
 'thresh_CNN_noisy': 0.5,
 'thresh_fitness_delta': -50,
 'thresh_fitness_raw': -60.97977932734429,
 'thresh_overlap': 0.5,
 'update_freq': 200,
 'update_num_comps': True,
 'use_corr_img': False,
 'use_dense': True,
 'use_peak_max': True,
 'W_update_factor': 1,
 'SNR_lowest': 0.5,
 'cnn_lowest': 0.1,
 'gSig_range': None,
 'min_cnn_thr': 0.9,
 'rval_lowest': -1,
 'use_cnn': False,
 'use_ecc': False,
 'max_ecc': 3,
 'do_merge': True,
 'merge_thr': 0.8,
 'merge_parallel': False,
 'max_merge_area': None,
 'border_nan': 'copy',
 'gSig_filt': None,
 'is3D': False,
 'max_deviation_rigid': 3,
 'max_shifts': (6, 6, 1),
 'min_mov': None,
 'niter_rig': 1,
 'nonneg_movie': True,
 'num_frames_split': 80,
 'num_splits_to_process_els': None,
 'num_splits_to_process_rig': None,
 'overlaps': (32, 32, 1),
 'pw_rigid': False,
 'shifts_opencv': True,
 'splits_els': 14,
 'splits_rig': 14,
 'strides': (96, 96, 1),
 'upsample_factor_grid': 4,
 'use_cuda': False,
 'n_channels': 2,
 'use_bias': False,
 'use_add': False,
 'pct': 0.01,
 'patience': 3,
 'max_epochs': 100,
 'width': 5,
 'loss_fn': 'pct',
 'lr': 0.001,
 'lr_scheduler': None,
 'remove_activity': False,
 'reuse_model': False}

### Insert CaImAn parameters

In [7]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method='caiman', 
    package_version='1.8.7', 
    paramset_idx=2, 
    paramset_desc='Calcium imaging analysis with CaImAn using default CaImAn parameters for 3d volumetric images', 
    params=params_caiman_3d)

## Insert entries into the following manual DataJoint tables

+ subject.Subject
+ experiment.Equipment
+ experiment.Session
+ scan.Scan
+ imaging.ProcessingTask

As described in the following sections, the entries can be inserted manually, or with the `ingest` method and accompanying `csv` files.

In [12]:
subject.Subject.insert1(dict(subject='subject1', 
                             sex='F', 
                             subject_birth_date='2019-01-01 00:00:01', 
                             subject_description='no description'))

Equipment.insert1(dict(scanner='MINI2P_ImagingScanner'))

Session.insert1(dict(subject='subject1', 
                     session_datetime='2021-01-01 00:00:01'))

Session.Directory.insert1(dict(subject='subject1', 
                               session_datetime='2021-01-01 00:00:01', 
                               session_dir='<imaging_root_data_dir>/subject1/session0'))

scan.Scan.insert1(dict(subject='subject1', 
                       session_datetime='2021-01-01 00:00:01', 
                       scan_id=0, 
                       scanner='MINI2P_ImagingScanner', 
                       scan_notes=''))

imaging.ProcessingTask.insert1(dict(subject='subject1', 
                                    session_datetime='2021-01-01 00:00:01', 
                                    scan_id=0, 
                                    paramset_idx=0, 
                                    task_mode='load'))

## Populate the following imported and computed DataJoint tables

+ scan.ScanInfo
+ imaging.Processing
+ imaging.MotionCorrection
+ imaging.Segmentation
+ imaging.MaskClassification
+ imaging.Fluorescence
+ imaging.Activity

As described in the following sections, populating the tables can be done individually or with the `populate` method.

In [ ]:
populate_settings = {'display_progress': True, 'reserve_jobs': False, 'suppress_errors': False}

scan.ScanInfo.populate(**populate_settings)
imaging.Processing.populate(**populate_settings)
imaging.MotionCorrection.populate(**populate_settings)
imaging.Segmentation.populate(**populate_settings)
imaging.MaskClassification.populate(**populate_settings)
imaging.Fluorescence.populate(**populate_settings)
imaging.Activity.populate(**populate_settings)

## Insert new ProcessingTask to trigger ingestion of motion-correction/segmentation results

Motion correction and segmentation are performed for each scan,
once the processing job has been completed, an entry in the `ProcessingTask` needs to be added to trigger the ingestion of the processing results.

Two pieces of information need to be specified:
+ the `paramset_idx` used for the processing job
+ the output directory storing the processing results

In [ ]:
scan.Scan()

In [ ]:
import pathlib
from workflow_imaging.paths import get_imaging_root_data_dir

root_dir = pathlib.Path(get_imaging_root_data_dir())

In [ ]:
for scan_key in (scan.Scan & scan.ScanInfo - imaging.ProcessingTask).fetch('KEY'):
    scan_file = root_dir / (scan.ScanInfo.ScanFile & scan_key).fetch('file_path')[0]
    recording_dir = scan_file.parent
    # suite2p
    suite2p_dir = recording_dir / 'suite2p'
    if suite2p_dir.exists():
        imaging.ProcessingTask.insert1({**scan_key,
                                        'paramset_idx': 0,
                                        'processing_output_dir': suite2p_dir.as_posix()})
    # caiman
    caiman_dir = recording_dir / 'caiman'
    if caiman_dir.exists():
        is_3D = (scan.ScanInfo & scan_key).fetch1('ndepths') > 1
        imaging.ProcessingTask.insert1({**scan_key,
                                        'paramset_idx': 1 if not is_3D else 2,
                                        'processing_output_dir': caiman_dir.as_posix()})

In [ ]:
imaging.ProcessingTask()

## Insert new Curation following the ProcessingTask

In this example, we create/insert one Curation for each ClusteringTask, specifying the same output directory

To this end, we make use of a convenient function `imaging.Curation().create1_from_processing_task()`

In [ ]:
for key in (imaging.ProcessingTask - imaging.Curation).fetch('KEY'):
    imaging.Curation().create1_from_processing_task(key)